In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset, random_split

In [7]:
# Load and preprocess dataset
train_df = pd.read_csv("Dataset/mitbih_train.csv", header=None)
test_df = pd.read_csv("Dataset/mitbih_test.csv", header=None)

# Binary classification: Convert class labels to 0 and 1
train_df[187] = train_df[187].apply(lambda x: 0 if x == 0 else 1)
test_df[187] = test_df[187].apply(lambda x: 0 if x == 0 else 1)

X_train = train_df.iloc[:, :-1].values
y_train = train_df[187].values
X_test = test_df.iloc[:, :-1].values
y_test = test_df[187].values

In [9]:
print(train_df)

            0         1         2         3         4         5         6    \
0      0.977941  0.926471  0.681373  0.245098  0.154412  0.191176  0.151961   
1      0.960114  0.863248  0.461538  0.196581  0.094017  0.125356  0.099715   
2      1.000000  0.659459  0.186486  0.070270  0.070270  0.059459  0.056757   
3      0.925414  0.665746  0.541436  0.276243  0.196133  0.077348  0.071823   
4      0.967136  1.000000  0.830986  0.586854  0.356808  0.248826  0.145540   
...         ...       ...       ...       ...       ...       ...       ...   
87549  0.807018  0.494737  0.536842  0.529825  0.491228  0.484211  0.456140   
87550  0.718333  0.605000  0.486667  0.361667  0.231667  0.120000  0.051667   
87551  0.906122  0.624490  0.595918  0.575510  0.530612  0.481633  0.444898   
87552  0.858228  0.645570  0.845570  0.248101  0.167089  0.131646  0.121519   
87553  0.901506  0.845886  0.800695  0.748552  0.687138  0.599073  0.512167   

            7         8         9    ...  178  179 

In [11]:
# Normalize and convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  # Adding channel dimension
X_test = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# Create DataLoaders
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [13]:
# Simulate hospitals by splitting the training data
hospital_1_size = len(train_dataset) // 3
hospital_2_size = len(train_dataset) // 3
hospital_3_size = len(train_dataset) - hospital_1_size - hospital_2_size

hospital_1_data, hospital_2_data, hospital_3_data = random_split(
    train_dataset, [hospital_1_size, hospital_2_size, hospital_3_size]
)

hospital_1_loader = DataLoader(hospital_1_data, batch_size=32, shuffle=True)
hospital_2_loader = DataLoader(hospital_2_data, batch_size=32, shuffle=True)
hospital_3_loader = DataLoader(hospital_3_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [15]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=1)
        
        # The output size after convolutions and pooling
        # Input size = 187
        # After conv1: (187 + 2*1 - 3) / 1 + 1 = 187
        # After pool: 187 / 2 = 93
        # After conv2: (93 + 2*1 - 3) / 1 + 1 = 93
        # After pool: 93 / 2 = 46 (rounded down)
        flattened_size = 64 * 46  # 64 channels * 46 length
        
        self.fc1 = nn.Linear(flattened_size, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # Convolution + Pooling
        x = self.pool(torch.relu(self.conv2(x)))  # Convolution + Pooling
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = torch.relu(self.fc1(x))  # Fully connected layer 1
        x = self.fc2(x)  # Fully connected layer 2
        return x

In [17]:
# Train the model on a single hospital
def train_on_hospital(model, data_loader, optimizer, criterion):
    model.train()
    for batch_idx, (data, target) in enumerate(data_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

In [19]:
# Federated training
def federated_training(model, hospitals, epochs=5):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}")
        for i, hospital_loader in enumerate(hospitals, 1):
            print(f"Training on Hospital {i}")
            train_on_hospital(model, hospital_loader, optimizer, criterion)

# Evaluate the model
def evaluate_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in data_loader:
            output = model(data)
            _, predicted = torch.max(output, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    accuracy = 100 * correct / total
    return accuracy

In [21]:
# Initialize the model
model = CNNModel()

In [23]:
# Train the model using federated learning
hospitals = [hospital_1_loader, hospital_2_loader, hospital_3_loader]
federated_training(model, hospitals, epochs=5)

Epoch 1
Training on Hospital 1
Training on Hospital 2
Training on Hospital 3
Epoch 2
Training on Hospital 1
Training on Hospital 2
Training on Hospital 3
Epoch 3
Training on Hospital 1
Training on Hospital 2
Training on Hospital 3
Epoch 4
Training on Hospital 1
Training on Hospital 2
Training on Hospital 3
Epoch 5
Training on Hospital 1
Training on Hospital 2
Training on Hospital 3


In [25]:
# Evaluate the global model
accuracy = evaluate_model(model, test_loader)
print(f"Accuracy on test set: {accuracy}%")

Accuracy on test set: 98.00840489676594%


In [ ]:
# Save the global model
torch.save(model.state_dict(), "federated_global_model.pth")
print("Federated Global Model saved as 'federated_global_model.pth'")